In [ ]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import math

from mpl_toolkits import mplot3d

pi = 3.1415

import csv

In [ ]:
def generate_initial_chromosomes(length, max, min, pop_size):
    return [[random.uniform(min,max) for j in range(length)] for i in range(pop_size)]

def population_stats(fitnesses):
    best = fitnesses[0]
    average = sum(fitnesses) / len(fitnesses)

    return best, average

In [ ]:
def rank_chromosomes(cost_function, chromosomes):

    fitnesses = []
    for chromosome in chromosomes:
        fitness = cost_function(chromosome)
        fitnesses.append(fitness)

    chromosome_with_fitnesses = []
    for i in range(len(chromosomes)):
        chromosome_with_fitnesses.append((chromosomes[i], fitnesses[i]))

    chromosome_with_fitnesses.sort(key=lambda x: x[1], reverse = True)

    sorted_chromosomes = []
    sorted_fitnesses = []
    for chromosome, cost in chromosome_with_fitnesses:
        sorted_chromosomes.append(chromosome)
        sorted_fitnesses.append(cost)

    return sorted_chromosomes, sorted_fitnesses

#SELECTION




##Roulette Wheel Selection

In [ ]:
def roulette_wheel_selection(parents, fitnesses):

    fitnesses_adjusted = [-1/x for x in fitnesses]

    num_parents = len(parents)

    pairs_count = num_parents // 2

    total_fitness = sum(fitnesses_adjusted)
    pairs = []

    for _ in range(pairs_count):
        selected_parents = []
        for _ in range(2):
            selection_point = random.uniform(0, total_fitness)
            running_sum = 0

            for parent, fitness in zip(parents, fitnesses_adjusted):
                running_sum += fitness
                if running_sum >= selection_point:
                    selected_parents.append(parent)
                    break

        pairs.append(selected_parents)

    return pairs

## Stochastic Uniform Selection

In [ ]:
def stochastic_uniform_selection(parents, fitnesses):

    fitnesses_adjusted = [-1/x for x in fitnesses]

    num_parents = len(parents)
    total_fitness = sum(fitnesses_adjusted)
    num_pairs = num_parents // 2
    distance = total_fitness / num_parents
    start_point = random.uniform(0, distance)
    points = [start_point + i * distance for i in range(num_parents)]

    selected_parents = []
    current_member = 0
    sum_fitness = fitnesses_adjusted[current_member]

    for point in points:
        while sum_fitness < point:
            current_member += 1
            sum_fitness += fitnesses_adjusted[current_member]
        selected_parents.append(parents[current_member])

    random.shuffle(selected_parents)

    pairs = []
    for i in range(0, len(selected_parents), 2):
        pairs.append(list((selected_parents[i], selected_parents[i+1])))

    return pairs

## K-tournament Selection

In [ ]:
def k_tournament_selection(cost_function, chromosomes, k):

    num_parents = len(chromosomes)
    num_pairs = num_parents // 2

    pairs = []

    for _ in range(0,num_pairs):
        pair = []
        for _ in range(2):
            tournament_contenders = random.choices(chromosomes, k=k)

            fitnesses = []

            for contender in tournament_contenders:
                fitness = cost_function(contender)
                fitnesses.append(fitness)

            pair.append(tournament_contenders[fitnesses.index(max(fitnesses))])
        pairs.append(pair)

    return pairs

##Linear Sorting Selection

In [ ]:
def linear_sorting_selection(chromosomes):

    N = len(chromosomes)
    num_pairs = N // 2
    probabilities = [(2 * (N - i)) / (N * (N + 1)) for i in range(N)]

    pairs = []
    for _ in range(num_pairs):
        selected_parents = random.choices(chromosomes, weights=probabilities, k=2)
        pairs.append(list(selected_parents))

    return pairs

## Aging Evolution

In [ ]:
def aging_evolution(chromosomes, sample_size, cost_function):

    pair = []
    for _ in range(2):
        tournament_contenders = random.choices(chromosomes, k=sample_size)

        fitnesses = []

        for contender in tournament_contenders:
            fitness = cost_function(contender)
            fitnesses.append(fitness)

        pair.append(tournament_contenders[fitnesses.index(max(fitnesses))])

    return pair

#CROSSOVER

In [ ]:
def crossover(pair):

    children = []

    p1, p2 = pair[0], pair[1]

    r = random.random()

    c1 = []
    c2 = []

    for i in range(0,len(p1)):
        c1.append(r * p1[i] + (1 - r) * p2[i])
        c2.append((1 - r) *  p1[i] + r*p2[i])

    children.append(c1)
    children.append(c2)

    return children

#MUTATION

In [ ]:
def mutation(chromosomes, mutation_rate, mutation_width):

    mutated_chromosomes = []
    for chromosome in chromosomes:

        result = []

        for i in range(0,len(chromosome)):
            if random.random() < mutation_rate:
                r = random.random()

                result.append( chromosome[i] + mutation_width * 2 * (r - 0.5) )
            else:
                result.append(chromosome[i])

        mutated_chromosomes.append(result)
    return mutated_chromosomes

#ELITISM

In [ ]:
def elitism(chromosomes_old, chromosomes_new, elitism_rate, population_size):

    old_to_keep = int(round(population_size * elitism_rate))

    elite_old_chromosomes = chromosomes_old[:old_to_keep]

    new_chromosomes = chromosomes_new[:population_size - old_to_keep]

    return elite_old_chromosomes + new_chromosomes

#TEST FUNCTIONS


In [ ]:
def levy_function(chromosome):

  x = chromosome[0]
  y = chromosome[1]

  tmp1 = math.pow(math.sin(3*pi*x), 2)
  tmp2 = math.pow((x - 1), 2) * (1 + math.pow(math.sin(3*pi*y), 2))
  tmp3 = math.pow((y - 1), 2) * (1 + math.pow(math.sin(2*pi*y), 2))

  return -(tmp1 + tmp2 + tmp3)

In [ ]:
def ackley_function(chromosome, a=20, b=0.2, c=2*math.pi):

    x = chromosome[0]
    y = chromosome[1]

    part1 = -a * math.exp(-b * math.sqrt(0.5 * (x**2 + y**2)))
    part2 = -math.exp(0.5 * (math.cos(c * x) + math.cos(c * y)))
    result = part1 + part2 + a + math.exp(1)
    return -result

In [ ]:
def sphere_function(chromosome):

    x = chromosome[0]
    y = chromosome[1]

    return -(x**2 + y**2)

In [ ]:
def rastrigin_function(chromosome, A=10):

    x = chromosome[0]
    y = chromosome[1]

    return -(A * 2 + (x**2 - A * math.cos(2 * math.pi * x)) + (y**2 - A * math.cos(2 * math.pi * y)))


#MAIN FUNCTION

In [ ]:
def genetic_algorithm(cost_function, range_of_values, population_size, selection_type, mutation_rate = 0.8, elitism_rate = 0.1, chromosome_length = 2, max_iterations = 500, function_tolerance = 0.005, pnt=False):

    min_val = range_of_values[0]
    max_val = range_of_values[1]

    avg_list = []
    best_list = []
    curr_best = -10000
    same_best_count = 0
    last_best = -100000
    stall_generations = 0

    chromosomes = generate_initial_chromosomes(chromosome_length, max_val, min_val, population_size)

    for iter in range(max_iterations):
      ranked_parents, costs = rank_chromosomes(cost_function, chromosomes)
      best, average = population_stats(costs)

      if selection_type == "roulette_wheel_selection":
          pairs = roulette_wheel_selection(ranked_parents, costs)
      elif selection_type == "stochastic_uniform_selection":
          pairs = stochastic_uniform_selection(ranked_parents, costs)
      elif selection_type == "k_tournament_selection":
          pairs = k_tournament_selection(cost_function, ranked_parents, 3)
      else:
          pairs = linear_sorting_selection(ranked_parents)

      if selection_type == "aging_evolution":
        pair = aging_evolution(chromosomes, 3, cost_function)
        child = crossover(pair)
        mutated_child = mutation(list(child), mutation_rate, 1)
        chromosomes.append(mutated_child[0])
        chromosomes.append(mutated_child[1])
        chromosomes.pop(0)
        chromosomes.pop(0)
      else:

        children = []

        for pair in pairs:
            children.extend(crossover(pair))

        mutated_children = mutation(children, mutation_rate, 1)

        ranked_children, costs = rank_chromosomes(cost_function, mutated_children)
        chromosomes = elitism(ranked_parents,ranked_children, elitism_rate, population_size)

      if(pnt):
        print("Generacija: ", iter+1," Prosek: {:.3f}".format(average)," Trenutno najbolje rešenje: {:.3f}".format(best), "[X, Y] = {:.3f} {:.3f}".format(ranked_parents[0][0], ranked_parents[0][1]))
        print("-------------------------")

      avg_list.append(average)
      if best > curr_best:
        best_list.append(best)
        last_best = curr_best
        curr_best = best
        same_best_count = 0
      elif best == curr_best:
        same_best_count += 1
        best_list.append(best)

      if (abs(abs(last_best) - abs(curr_best)) < function_tolerance):
          stall_generations = stall_generations + 1

          if stall_generations == 10:
              avg_list = avg_list[:iter]
              best_list = best_list[:iter]
              all_avg_list.append(avg_list)
              all_best_list.append(best_list)
              generations_list.append(iter)

              if(pnt):
                print("\nPronađeno rešenje! Koordinate rešenja: [X, Y] = {:.3f} {:.3f}\n".format(ranked_parents[0][0],ranked_parents[0][1]))
              return round(cost_function([ranked_parents[0][0],ranked_parents[0][1]]), 3)
      else:
          stall_generations = 0

      if same_best_count > 10:
        if(pnt):
          print("\nZaustavljeno zbog konvergencije. Najbolje rešenje: [X, Y] = {:.3f} {:.3f}\n".format(ranked_parents[0][0],ranked_parents[0][1]))

        avg_list = avg_list[:iter]
        best_list = best_list[:iter]
        all_avg_list.append(avg_list)
        all_best_list.append(best_list)
        generations_list.append(iter)

        return round(cost_function([ranked_parents[0][0],ranked_parents[0][1]]), 3)

      if iter == max_iterations - 1:
        avg_list = avg_list[:iter]
        best_list = best_list[:iter]
        all_avg_list.append(avg_list)
        all_best_list.append(best_list)
        generations_list.append(iter)

        if(pnt):
          print("\nZaustavljeno zbog dostignutog maksimalnog broja iteracija, nije pronađeno rešenje. Najbolje rešenje [X, Y] = {:.3f} {:.3f}\n".format(ranked_parents[0][0],ranked_parents[0][1]))

        return round(cost_function([ranked_parents[0][0],ranked_parents[0][1]]), 3)

In [ ]:
all_avg_list = []
generations_list = []
all_best_list = []
print(genetic_algorithm(levy_function, [-5, 5], 50, "roulette_wheel_selection", function_tolerance = 0.002, pnt=True))

Generacija:  1  Prosek: -29.604  Trenutno najbolje rešenje: -0.616 [X, Y] = 1.696 0.964
-------------------------
Generacija:  2  Prosek: -6.164  Trenutno najbolje rešenje: -0.125 [X, Y] = 1.334 0.964
-------------------------
Generacija:  3  Prosek: -1.791  Trenutno najbolje rešenje: -0.122 [X, Y] = 1.033 0.871
-------------------------
Generacija:  4  Prosek: -1.268  Trenutno najbolje rešenje: -0.071 [X, Y] = 1.026 0.916
-------------------------
Generacija:  5  Prosek: -1.070  Trenutno najbolje rešenje: -0.055 [X, Y] = 0.976 0.934
-------------------------
Generacija:  6  Prosek: -1.124  Trenutno najbolje rešenje: -0.001 [X, Y] = 0.997 1.019
-------------------------
Generacija:  7  Prosek: -0.837  Trenutno najbolje rešenje: -0.001 [X, Y] = 0.997 0.993
-------------------------
Generacija:  8  Prosek: -0.750  Trenutno najbolje rešenje: -0.001 [X, Y] = 0.997 0.993
-------------------------
Generacija:  9  Prosek: -0.686  Trenutno najbolje rešenje: -0.001 [X, Y] = 0.997 0.993
--------